In [197]:
class Node:
    
    def __init__(self, data):
        self.data = data
        self.height = 0
        self.left = None
        self.right = None


class AVLtree:
    
    def __init__(self, key=lambda x: x):
    
        self.root = None
        self.size = 0
        self.key = key
    
    def _getheight(self, node):
        '''return height of the node'''
        
        # if node == None; return -1
        if node:
            return node.height
        
        return -1
    
    def _updateheight(self, node):
        '''private method to update height'''
        
        node.height = max(self._getheight(node.left), self._getheight(node.right)) + 1
        
    def _heightdiff(self, node):
        '''returns diffrence in height of left and right children, +ve means left heavy'''
        
        if not node:
            return 0
        
        return self._getheight(node.left) - self._getheight(node.right)
    
    def _rightrotation(self, node):
        '''rotate right the node'''
        
        # make node node.left.right, and make node.left node.left.right
        temp = node.left  # new root of subtree
        node.left = node.left.right
        temp.right = node
        
        # update heights
        self._updateheight(node)
        self._updateheight(temp)
        
        return temp
    
    def _leftrotation(self, node):
        '''rotate left the node'''
        
        temp = node.right  # new root of subtree
        node.right = node.right.left
        temp.left = node
        
        # update heights
        self._updateheight(node)
        self._updateheight(temp)
        
        return temp
    
    def insert(self, data):
        '''insert item into the AVL tree'''
        
        if self.root:
            self.root = self._insert(data, self.root)
            
        else:
            # if this is first item into the tree
            self.root = Node(data)
            
        self.size += 1
    
    def _insert(self, data, node):
        '''private method to insert item'''
        
        if not node:
            node = Node(data)
        
        elif self.key(data) >= self.key(node.data):
            node.right = self._insert(data, node.right)
        
        else:
            node.left = self._insert(data, node.left)
            
        # update height
        self._updateheight(node)
        
        return self._makebalanced(data, node)
    
    def _makebalanced(self, data, node):
        '''private method to make tree AVL/balanced'''
        
        # check unbalance type
        if self._heightdiff(node) > 1:  #left heavy unbalance
            
            if self.key(data) < node.data:   # left left unbalance (new node has been added to the left of left child of node)
                return self._rightrotation(node)
            
            else:
                node.left = self._leftrotation(node.left)
                return self._rightrotation(node)
        
        elif self._heightdiff(node) < -1:  # right heavy
            
            if self.key(data) >= node.data:  # right right unbalance
                return self._leftrotation(node)
            
            else:
                node.right = self._rightrotation(node.right)
                return self._leftrotation(node)
        else:
            return node
    
    def remove(self, data):
        '''removes item from the bst'''
        
        if self.size == 0:
            raise Exception("Cannot remove from empty BST")
            
        self.root = self._remove(data, self.root)
        self.size -= 1
    
    def _remove(self, data, node):
        '''private method to remove item'''
        
        if not node:
            raise Exception("Item Not Found")
            
        # check if item is in right or left subtree
        if self.key(data) >= self.key(node.data) and data != node.data:  # even if key is equal, item can be diffrent
            # right child of node needs to be changed
            node.right = self._remove(data, node.right)
            
        elif self.key(data) < self.key(node.data):
            node.left = self._remove(data, node.left)
        
        # if item is equal to node.data
        else:
            # check if left node
            if not node.right and not node.left:
                del node
                return None   # parent node should have None child
            # if only left child 
            if not node.right:
                return node.left  # parent should have node.left as child
            # only right child
            if not node.left:
                return node.right
            # node has both children
            # replace node.data with data of its predecessor or sucessor
            predecessor = self._getpredecessor(node)
            node.data = predecessor.data
            # now remove predecessor which will be easy because it will have atmost one child i.e. left child since it is rightmost node
            node.left = self._remove(predecessor.data, node.left) # root for this operation is node.left
            
        if not node:
            return
        
        # update height
        self._updateheight(node)
        
        if self._heightdiff(node) > 1:  # left heavy
            
            if self._heightdiff(node.left) >= 0: # left left 
                return self._rightrotation(node)
            else:
                node.left = self._leftrotation(node.left)
                return node._rightrotation(node)
            
        elif self._heightdiff(node) < - 1: # right heavy
            
            if self._heightdiff(node.right) <= 0:
                return self._leftrotation(node)
            else:
                node.right = self._rightrotation(node.right)
                return self._leftrotation(node)
            
        return node
    
    def _getpredecessor(self, node):
        '''private method to get predecessor i.e. max node of left subtree'''
        
        # get rightmost node of subtree for curr
        return self._getmax(node.left)
    
    def getmin(self):
        '''return minimum item '''
        
        return self._getmin().data
    
    def _getmin(self, curr=None):
        '''private method to return leftmot node'''
        
        if self.size < 1:
              raise Exception("No item in the BST")
        if not curr:
            curr = self.root
        # get leftmost node
        while curr.left:
            curr = curr.left
        return curr
    
    def getmax(self):
        '''return maximum item'''
        
        return self._getmax().data
    
    def _getmax(self, curr=None):
        '''private method to return rightmost node'''
        
        if self.size < 1:
            raise Exception("No item in the BST")
        if not curr:  
            curr = self.root
        # get rightmost node
        while curr.right:
            curr = curr.right
            
        return curr
    
    def getsorted(self):
        '''returns list of items in non decreasing order (comparison via key)'''
        
        out = []
        if self.size == 0:
            return out
        
        self._inorder(self.root, out)
        return out
    
    def _inorder(self, node, arr):
        '''adds items in the arr in order'''
        
        # recusivly add nodes in order ....left, root, right
        if node.left:
            self._inorder(node.left, arr)
            
        arr.append(node.data)
        
        if node.right:
            self._inorder(node.right, arr)

In [221]:
avl = AVLtree()

In [222]:
l = [12,43,12,46,67,12,10,100]
for i in l:
    avl.insert(i)

In [223]:
avl.size

8

In [224]:
avl.getsorted()

[10, 12, 12, 12, 43, 46, 67, 100]

In [225]:
avl.getmax()

100

In [226]:
avl.getmin()

10

In [227]:
avl.remove(67)

In [228]:
avl.getsorted()

[10, 12, 12, 12, 43, 46, 100]